In [22]:
import pandas as pd
from sqlalchemy import create_engine

In [23]:
#Extract wine_data CSV into DataFrame (data source #1)
wine_file = "../wine_data.csv"
wine_df = pd.read_csv(wine_file)
wine_df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [24]:
#Transform wine_data DataFrame

transformed_wine_df = wine_df.drop(columns=['taster_name', 'taster_twitter_handle','Unnamed: 0','price','description','designation','title'])
new_transformed_wine_df = transformed_wine_df.loc[transformed_wine_df['country'] == 'US']
new_transformed_wine_df = new_transformed_wine_df.dropna()
new_transformed_wine_df = new_transformed_wine_df.reset_index()
us_transformed_wine_df = new_transformed_wine_df.drop(columns=['index','country'])
us_transformed_wine_df = us_transformed_wine_df.rename(columns={'region_1':'Wine Region','region_2' : 'Sub-Region'})
us_transformed_wine_df["Lat"] = ""
us_transformed_wine_df["Lng"] = ""
us_transformed_wine_df

,points,province,Wine Region,Sub-Region,variety,winery,Lat,Lng
0,87,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm,,
1,87,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks,,
2,87,California,Napa Valley,Napa,Cabernet Sauvignon,Kirkland Signature,,
3,87,California,Alexander Valley,Sonoma,Cabernet Sauvignon,Louis M. Martini,,
4,87,California,Central Coast,Central Coast,Chardonnay,Mirassou,,
...,...,...,...,...,...,...,...,...
50506,90,California,Santa Ynez Valley,Central Coast,Chenin Blanc,Birichino,,
50507,90,California,Napa Valley,Napa,Chardonnay,Flora Springs,,
50508,90,California,Napa Valley,Napa,Zinfandel,Hendry,,
50509,90,California,Chiles Valley,Napa,Zinfandel,Houdini,,


In [39]:
state_df = us_transformed_wine_df.groupby(['province']).count()
state_df["Lat"] = ""
state_df["Lng"] = ""
state_df = state_df.reset_index()
state_df

,province,points,Wine Region,Sub-Region,variety,winery,Lat,Lng
0,California,34119,34119,34119,34119,34119,,
1,New York,2688,2688,2688,2688,2688,,
2,Oregon,5067,5067,5067,5067,5067,,
3,Washington,8637,8637,8637,8637,8637,,


In [40]:
# use google API to add the zip code by winery
from config import gkey
import requests
import json

params = {
    "inputtype": "textquery&fields=formatted_address,name,rating,geometry",
    "key": gkey,
}

for index, row in state_df.iterrows():
    # get winery from df
    state = row["province"]
    
    params['input'] = state
    
    base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

    # make request and print url
    state_google = requests.get(base_url, params=params)

    # print(winery_zipcode.url)

    # convert to json
    state_google = state_google.json()
    # print(json.dumps(state_google, indent=4, sort_keys=True))

    state_df.loc[index, "Lat"] = state_google["results"][0]['geometry']['location']["lat"]
    state_df.loc[index, "Lng"] = state_google["results"][0]['geometry']['location']["lng"]


In [41]:
state_df = state_df.drop(columns=['points', 'Wine Region','Sub-Region','variety','winery'])
state_df

,province,Lat,Lng
0,California,36.7783,-119.418
1,New York,40.7128,-74.006
2,Oregon,43.8041,-120.554
3,Washington,47.7511,-120.74


In [44]:
wine_state_df = us_transformed_wine_df.merge(state_df, on='province')
wine_state_df = wine_state_df.drop(columns=['Lat_x', 'Lng_x'])
wine_state_df = wine_state_df.rename(columns={'points':'Total Points','province' : 'Province','variety' : 'Variety','winery' : 'Winery','Lat_y' : 'Lat','Lng_y' : 'Lng'})
wine_state_df

,Total Points,Province,Wine Region,Sub-Region,Variety,Winery,Lat,Lng
0,87,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm,43.8041,-120.554
1,87,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks,43.8041,-120.554
2,87,Oregon,Oregon,Oregon Other,Pinot Noir,Acrobat,43.8041,-120.554
3,86,Oregon,McMinnville,Willamette Valley,Pinot Noir,Erath,43.8041,-120.554
4,86,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Hawkins Cellars,43.8041,-120.554
...,...,...,...,...,...,...,...,...
50506,89,New York,Finger Lakes,Finger Lakes,Pinot Gris,Dr. Konstantin Frank,40.7128,-74.006
50507,89,New York,Finger Lakes,Finger Lakes,Sauvignon Blanc,Hazlitt 1852 Vineyards,40.7128,-74.006
50508,88,New York,Seneca Lake,Finger Lakes,Riesling,Belhurst,40.7128,-74.006
50509,88,New York,Finger Lakes,Finger Lakes,Pinot Grigio,Penguin Bay,40.7128,-74.006


In [49]:
connection_string = "postgres:postgres@localhost:5432/wasted_away_in_wine_land"
engine = create_engine(f'postgresql://{connection_string}')

engine.table_names()
wine_state_df.to_sql(name='wine_state', con=engine, if_exists='append', index=True)
state_df.to_sql(name='state lat_lng', con=engine, if_exists='append', index=True)
us_transformed_wine_df.to_sql(name='winery stats', con=engine, if_exists='append', index=True)

In [46]:
wine_state_df.to_csv("wine_state.csv", index=False, header=True)